In [3]:
import os
import cv2
import numpy as np
import scipy.io as sio
import torch
from mapanything.models import MapAnything
from mapanything.utils.image import load_images

In [4]:
# Get inference device
device = "cpu" 

# Init model - This requires internet access or the huggingface hub cache to be pre-downloaded
# For Apache 2.0 license model, use "facebook/map-anything-apache"
model = MapAnything.from_pretrained("facebook/map-anything").to(device)

# Load and preprocess images from a folder or list of paths
#images = ["/Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/Datasets/TaagPIV/20251028_165814.jpg" ] # or ["path/to/img1.jpg", "path/to/img2.jpg", ...]
images = "/Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/AulasPraticas/aula7_3D_registration/plondres" # or ["path/to/img1.jpg", "path/to/img2.jpg", ...]
views = load_images(images)

Loading pretrained dinov2_vitl14 from torch hub


Using cache found in /Users/jpc/.cache/torch/hub/facebookresearch_dinov2_main


In [15]:

# ============================================================
# 1. Scan image directory
# ============================================================

def scan_image_directory(
    image_dir,
    extensions=(".jpg", ".jpeg", ".png")
):
    """
    Scan directory and return sorted list of image paths.
    """
    image_paths = [
        os.path.join(image_dir, f)
        for f in os.listdir(image_dir)
        if f.lower().endswith(extensions)
    ]

    if not image_paths:
        raise ValueError(f"No images found in {image_dir}")

    image_paths.sort()
    return image_paths


# ============================================================
# 2. MapAnything Images-Only Inference
# ============================================================

def run_mapanything_images_only(image_paths):
    """
    Call MapAnything Images-Only Inference.
    """
    views = load_images(images)
    # Run inference
    predictions = model.infer(
        views,                            # Input views
        memory_efficient_inference=False, # Trades off speed for more views (up to 2000 views on 140 GB)
        use_amp=True,                     # Use mixed precision inference (recommended)
        amp_dtype="bf16",                 # bf16 inference (recommended; falls back to fp16 if bf16 not supported)
        apply_mask=True,                  # Apply masking to dense geometry outputs
        mask_edges=True,                  # Remove edge artifacts by using normals and depth
        apply_confidence_mask=False,      # Filter low-confidence regions
        confidence_percentile=10,         # Remove bottom 10 percentile confidence pixels
    )
        
    d=[]
    for i, pred in enumerate(predictions):
        d.append({
            key: value.detach().cpu().numpy().squeeze() for key, value in pred.items()
        })
    return(d)

# ============================================================
# 3. Utilities
# ============================================================

def _save_rgb_image(img, path):
    if img.ndim == 3 and img.shape[0] == 3:
        img = np.transpose(img, (1, 2, 0))

    img = img.astype(np.float32)
    if img.max() <= 1.0:
        img *= 255.0

    img = np.clip(img, 0, 255).astype(np.uint8)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, img_bgr)


def _save_depth_mat(depth, K, path):
    sio.savemat(
        path,
        {
            "depth": depth.astype(np.float32),
            "K": K.astype(np.float32),
        }
    )


# ============================================================
# 4. Template + sequence generation
# ============================================================

def generate_template(pred0, template_dir):
    os.makedirs(template_dir, exist_ok=True)

    _save_rgb_image(
        pred0["img_no_norm"],
        os.path.join(template_dir, "templatergb.jpg"),
    )

    _save_depth_mat(
        pred0["depth_z"],
        pred0["intrinsics"],
        os.path.join(template_dir, "templatedepth.mat"),
    )


def generate_sequence(preds, sequence_dir, base_name="frame"):
    os.makedirs(sequence_dir, exist_ok=True)

    for i, pred in enumerate(preds):
        idx_str = f"{i:04d}"

        _save_rgb_image(
            pred["img_no_norm"],
            os.path.join(sequence_dir, f"{base_name}_{idx_str}.jpg"),
        )

        _save_depth_mat(
            pred["depth_z"],
            pred["intrinsics"],
            os.path.join(sequence_dir, f"{base_name}_{idx_str}.mat"),
        )


# ============================================================
# 5. ONE-CALL dataset generation
# ============================================================

def generate_full_dataset(
    image_dir,
    output_root,
    base_name="frame",
):
    """
    Generate full assignment dataset from an image directory.
    """

    # ---- Scan directory ----
    image_paths = scan_image_directory(image_dir)

    # ---- Run MapAnything ----
    preds = run_mapanything_images_only(image_paths)

    if len(preds) < 2:
        raise ValueError("Need at least 2 images for template + sequence")

    # ---- Output structure ----
    template_dir = os.path.join(output_root, "template")
    sequence_dir = os.path.join(output_root, "sequence")
    ground_truth = os.path.join(output_root, "gtruth")
    os.makedirs(ground_truth, exist_ok=True)
    sio.savemat(os.path.join(ground_truth,"alldata.mat"),{"predicts":preds})
    
    # ---- Template from first frame ----
    generate_template(preds[0], template_dir)

    # ---- Sequence from remaining frames ----
    generate_sequence(
        preds=preds[1:],
        sequence_dir=sequence_dir,
        base_name=base_name,
    )

    print("Dataset generation complete")
    print(f"Template directory : {template_dir}")
    print(f"Sequence directory : {sequence_dir}")

In [16]:
generate_full_dataset(
    image_dir= "/Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/AulasPraticas/aula7_3D_registration/plondres/",
    output_root="/Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/AulasPraticas/aula7_3D_registration/plondres/plondres",
)

/Users/jpc/Nextcloud/IST/code/aisp/map-anything/mapanything/models/mapanything/model.py:2052: UserWarning: bf16 is not supported on this device. Using fp16 instead.
  warnings.warn(
/Users/jpc/piton/lib/python3.13/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Dataset generation complete
Template directory : /Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/AulasPraticas/aula7_3D_registration/plondres/plondres/template
Sequence directory : /Users/jpc/Nextcloud/IST/Escolaridade/PIV2025/AulasPraticas/aula7_3D_registration/plondres/plondres/sequence
